In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import random

from pycaret.classification import *
from sklearn.metrics import log_loss

In [2]:
seed = 42
np.random.seed(seed)

In [36]:
train = pd.read_csv('input/train.csv', index_col=['index'])
test = pd.read_csv('input/test.csv', index_col=['index'])
submission = pd.read_csv('input/submission.csv', index_col=['index'])

train.shape, test.shape, submission.shape

((26457, 19), (10000, 18), (10000, 3))

## null 처리 

In [37]:
print(8171/train.shape[0]*100)
display(train.isna().sum())

print(3152/test.shape[0]*100)
display(test.isna().sum())

30.884076047926822


gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64

31.52


gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       3152
family_size         0
begin_month         0
dtype: int64

In [38]:
train.drop(['occyp_type'], axis=1, inplace=True)
test.drop(['occyp_type'], axis=1, inplace=True)
train.shape, test.shape

((26457, 18), (10000, 17))

In [45]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,39,13,1,0,0,0,2.0,0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,31,4,1,0,0,1,3.0,0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,53,12,1,0,1,0,2.0,1,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,42,5,1,0,1,0,2.0,3,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,42,5,1,0,0,0,2.0,2,2.0


In [40]:
train.DAYS_BIRTH.value_counts()

-12676    40
-15519    38
-14667    32
-15140    26
-16768    24
          ..
-9353      1
-9901      1
-21655     1
-11767     1
-15726     1
Name: DAYS_BIRTH, Length: 6621, dtype: int64

In [41]:
train[['income_total']].sort_values('income_total')

,income_total
index,
6693,27000.0
11224,27000.0
4797,29250.0
18054,29250.0
15900,29250.0
...,...
18445,1575000.0
10564,1575000.0
14702,1575000.0


In [42]:
train.loc[(train.DAYS_EMPLOYED == 365243), 'DAYS_EMPLOYED'] = 0
test.loc[(test.DAYS_EMPLOYED == 365243), 'DAYS_EMPLOYED'] = 0

train['DAYS_EMPLOYED'] = list(map(int, -train['DAYS_EMPLOYED']/356))
test['DAYS_EMPLOYED'] = list(map(int, -test['DAYS_EMPLOYED']/356))

In [43]:
train['DAYS_BIRTH'] = list(map(int, -train['DAYS_BIRTH']/356))
test['DAYS_BIRTH'] = list(map(int, -test['DAYS_BIRTH']/356))

In [44]:
train['begin_month'] = list(map(int, -train.begin_month/12))
test['begin_month'] = list(map(int, -test.begin_month/12))

## outlier

## 수치형 자료

## 범주형 자료 

In [46]:
col = ['gender','car','reality','phone','email','work_phone'\
      ,'income_type','edu_type','family_type','house_type','begin_month'\
       #,'DAYS_BIRTH','DAYS_EMPLOYED'
      ]
#train = pd.get_dummies(train, columns=col)
#test = pd.get_dummies(test, columns=col)
train.shape, test.shape

((26457, 18), (10000, 17))

In [ ]:
train.columns

In [47]:
%%time
clf = setup(session_id=seed, 
            data = train, target = 'credit'
          , categorical_features=col
          , numeric_features = ['child_num','family_size']#,,'tp17','tp4','Q_A15','Q_A3','Q_A15tp17','Q_A3tp4']#['Q_A1','Q_A3','Q_A5','tp17','tp4','Q_A15tp17','Q_A3tp4']
          #, ignore_features =['FLAG_MOBIL','work_phone','phone']
           )

,Description,Value
0,session_id,42
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 18)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,12
8,Ordinal Features,False
9,High Cardinality Features,False


Wall time: 4.81 s


In [48]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function               <function log_loss at 0x000001ED7DC99558>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [49]:
#best5 = compare_models(fold = 5, sort = 'logloss', n_select = 5, exclude=['svm','ridge'])
best_3 = compare_models(sort = 'LogLoss', n_select = 3
                       ,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'ridge','qda','ada','et']
                       #,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'gpc', 'mlp', 'ridge', 'rf', 'qda', 'ada', 'xgboost']
                       )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
catboost,CatBoost Classifier,0.6628,0.6805,0.4118,0.6264,0.5974,0.1726,0.2108,0.8133,11.1490
lightgbm,Light Gradient Boosting Machine,0.6552,0.6693,0.3758,0.6289,0.5639,0.1061,0.1570,0.8206,0.3700
gbc,Gradient Boosting Classifier,0.6471,0.6283,0.3534,0.5831,0.5380,0.0560,0.1026,0.8462,2.5060
lda,Linear Discriminant Analysis,0.6438,0.6073,0.3584,0.5260,0.5458,0.0697,0.1071,0.8555,0.0700
rf,Random Forest Classifier,0.6462,0.6687,0.4507,0.6081,0.6140,0.2149,0.2261,1.1133,1.3600


In [50]:
blended = blend_models(estimator_list = best_3, fold = 3, optimize = 'logloss')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6549,0.6583,0.3777,0.6385,0.5673,0.1129,0.1600,0.8260
1,0.6520,0.6696,0.3747,0.6165,0.5622,0.1007,0.1468,0.8213
2,0.6569,0.6764,0.3809,0.6283,0.5702,0.1194,0.1686,0.8174
Mean,0.6546,0.6681,0.3778,0.6277,0.5666,0.1110,0.1584,0.8216
SD,0.0020,0.0075,0.0026,0.0090,0.0033,0.0077,0.0089,0.0036


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6466,0.6789,0.3735,0.6061,0.5541,0.1019,0.1501,0.8232


In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model])
prections = prep_pipe.predict_proba(test)
prections

In [ ]:
submission['0'] = prections[:,0]
submission['1'] = prections[:,1]
submission['2'] = prections[:,2]
submission

In [ ]:
submission.to_csv('output/20210405-4.csv')